In [43]:
# import / dependencies
import os
import tarfile

import pandas as pd

import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_squared_error

from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV

from six.moves import urllib

In [3]:
# variables
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("local_training_data", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

# function
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

fetch_housing_data()

In [4]:
# Analyze the structure
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [5]:
data_df = load_housing_data()
data_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [6]:
data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           20640 non-null  float64
 1   latitude            20640 non-null  float64
 2   housing_median_age  20640 non-null  float64
 3   total_rooms         20640 non-null  float64
 4   total_bedrooms      20433 non-null  float64
 5   population          20640 non-null  float64
 6   households          20640 non-null  float64
 7   median_income       20640 non-null  float64
 8   median_house_value  20640 non-null  float64
 9   ocean_proximity     20640 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [7]:
train_df, test_df = train_test_split(data_df, test_size=0.2, random_state=42)

In [8]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16512 entries, 14196 to 15795
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           16512 non-null  float64
 1   latitude            16512 non-null  float64
 2   housing_median_age  16512 non-null  float64
 3   total_rooms         16512 non-null  float64
 4   total_bedrooms      16512 non-null  float64
 5   population          16512 non-null  float64
 6   households          16512 non-null  float64
 7   median_income       16512 non-null  float64
 8   median_house_value  16512 non-null  float64
 9   ocean_proximity     16512 non-null  object 
dtypes: float64(9), object(1)
memory usage: 1.4+ MB


In [9]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4128 entries, 20046 to 3665
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           4128 non-null   float64
 1   latitude            4128 non-null   float64
 2   housing_median_age  4128 non-null   float64
 3   total_rooms         4128 non-null   float64
 4   total_bedrooms      3921 non-null   float64
 5   population          4128 non-null   float64
 6   households          4128 non-null   float64
 7   median_income       4128 non-null   float64
 8   median_house_value  4128 non-null   float64
 9   ocean_proximity     4128 non-null   object 
dtypes: float64(9), object(1)
memory usage: 354.8+ KB


In [10]:
# Separate the predictors and the labels
# train
x_train_df = train_df.drop("median_house_value", axis=1)
y_train_df = train_df["median_house_value"].copy()
# test
x_test_df = test_df.drop("median_house_value", axis=1)
y_test_df = test_df["median_house_value"].copy()

In [11]:
x_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16512 entries, 14196 to 15795
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   longitude           16512 non-null  float64
 1   latitude            16512 non-null  float64
 2   housing_median_age  16512 non-null  float64
 3   total_rooms         16512 non-null  float64
 4   total_bedrooms      16512 non-null  float64
 5   population          16512 non-null  float64
 6   households          16512 non-null  float64
 7   median_income       16512 non-null  float64
 8   ocean_proximity     16512 non-null  object 
dtypes: float64(8), object(1)
memory usage: 1.3+ MB


In [12]:
y_train_df.info()

<class 'pandas.core.series.Series'>
Index: 16512 entries, 14196 to 15795
Series name: median_house_value
Non-Null Count  Dtype  
--------------  -----  
16512 non-null  float64
dtypes: float64(1)
memory usage: 258.0 KB


In [13]:
numeric_features = ['longitude', 'latitude', 'housing_median_age', 'total_rooms', 'total_bedrooms', 'population', 'households', 'median_income']
categorical_features = ["ocean_proximity"]

In [14]:
# preprocessing pipeline definition
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy="mean")),
    ('scaler', StandardScaler()),
])

categorical_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy="most_frequent")),
    ('encoder', OneHotEncoder()),
])

preprocessor = ColumnTransformer([
    ("numeric", numeric_transformer, numeric_features),
    ("categorical", categorical_transformer, categorical_features),
])

preprocessing_pipeline = Pipeline([
    ("preprocessor", preprocessor),
])

In [15]:
x_train_df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
14196,-117.03,32.71,33.0,3126.0,627.0,2300.0,623.0,3.2596,NEAR OCEAN
8267,-118.16,33.77,49.0,3382.0,787.0,1314.0,756.0,3.8125,NEAR OCEAN
17445,-120.48,34.66,4.0,1897.0,331.0,915.0,336.0,4.1563,NEAR OCEAN
14265,-117.11,32.69,36.0,1421.0,367.0,1418.0,355.0,1.9425,NEAR OCEAN
2271,-119.80,36.78,43.0,2382.0,431.0,874.0,380.0,3.5542,INLAND


In [16]:
x_train = preprocessing_pipeline.fit_transform(x_train_df)
x_train

array([[ 1.27258656, -1.3728112 ,  0.34849025, ...,  0.        ,
         0.        ,  1.        ],
       [ 0.70916212, -0.87669601,  1.61811813, ...,  0.        ,
         0.        ,  1.        ],
       [-0.44760309, -0.46014647, -1.95271028, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 0.59946887, -0.75500738,  0.58654547, ...,  0.        ,
         0.        ,  0.        ],
       [-1.18553953,  0.90651045, -1.07984112, ...,  0.        ,
         0.        ,  0.        ],
       [-1.41489815,  0.99543676,  1.85617335, ...,  0.        ,
         1.        ,  0.        ]])

In [17]:
x_train_df2 = pd.DataFrame(x_train)
x_train_df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,1.272587,-1.372811,0.348490,0.222569,0.211228,0.768276,0.322906,-0.326196,0.0,0.0,0.0,0.0,1.0
1,0.709162,-0.876696,1.618118,0.340293,0.593094,-0.098901,0.672027,-0.035843,0.0,0.0,0.0,0.0,1.0
2,-0.447603,-0.460146,-1.952710,-0.342597,-0.495226,-0.449818,-0.430461,0.144701,0.0,0.0,0.0,0.0,1.0
3,1.232698,-1.382172,0.586545,-0.561490,-0.409306,-0.007434,-0.380587,-1.017864,0.0,0.0,0.0,0.0,1.0
4,-0.108551,0.532084,1.142008,-0.119565,-0.256559,-0.485877,-0.314962,-0.171488,0.0,1.0,0.0,0.0,0.0


In [18]:
x_test = preprocessing_pipeline.transform(x_test_df)
x_test

array([[ 0.28534728,  0.1951    , -0.28632369, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.06097472, -0.23549054,  0.11043502, ...,  0.        ,
         0.        ,  0.        ],
       [-1.42487026,  1.00947776,  1.85617335, ...,  0.        ,
         1.        ,  0.        ],
       ...,
       [-1.23041404,  0.78014149, -0.28632369, ...,  0.        ,
         0.        ,  0.        ],
       [-0.08860699,  0.52740357,  0.58654547, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.60445493, -0.66608108, -0.92113763, ...,  0.        ,
         0.        ,  0.        ]])

In [19]:
x_test_df2 = pd.DataFrame(x_test)
x_test_df2.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.285347,0.195100,-0.286324,-0.522862,0.0,-0.030301,-0.370087,-1.155085,0.0,1.0,0.0,0.0,0.0
1,0.060975,-0.235491,0.110435,0.138415,0.0,0.121851,0.220532,-0.708659,0.0,1.0,0.0,0.0,0.0
2,-1.424870,1.009478,1.856173,0.546310,0.0,-0.102419,1.215396,-0.210402,0.0,0.0,0.0,1.0,0.0
3,0.429943,-0.637999,-0.921138,0.188080,0.0,0.244979,-0.013091,0.975113,1.0,0.0,0.0,0.0,0.0
4,-1.170581,0.457199,0.427842,-0.133821,0.0,-0.319653,-0.188964,-0.081794,0.0,0.0,0.0,0.0,1.0


In [20]:
y_train = y_train_df.to_numpy()
y_train

array([103000., 382100., 172600., ..., 222100., 283500., 325000.])

In [21]:
y_test = y_test_df.to_numpy()
y_test

array([ 47700.,  45800., 500001., ..., 500001.,  72300., 151500.])

In [22]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(16512, 13)
(16512,)
(4128, 13)
(4128,)


In [24]:
# train using from LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)

LinearRegression()

In [25]:
# evaluate
x_val = x_test[:10]
y_val = y_test[:10]

In [26]:
y_pred = lin_reg.predict(x_val)

In [28]:
print(y_pred)
print(y_val)

[ 65056. 135456. 265888. 278176. 274304. 150784. 300896. 238496. 266912.
 418272.]
[ 47700.  45800. 500001. 218600. 278000. 158700. 198200. 157500. 340000.
 446600.]


In [31]:
# let's measure the regression using mse
preds = lin_reg.predict(x_test)
lin_mse = mean_squared_error(y_test, preds)
lin_rmse = np.sqrt(lin_mse)
lin_rmse
# prediction error of 70,023

70023.78098181603

In [33]:
# let's change the algorithm
tree_reg = DecisionTreeRegressor()
tree_reg.fit(x_train, y_train)

DecisionTreeRegressor()

In [34]:
# let's measure the tree regression using mse
preds = tree_reg.predict(x_test)
tree_mse = mean_squared_error(y_test, preds)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

69289.2198428197

In [36]:
# Evaluation Using Cross-Validation
scores = cross_val_score(tree_reg, x_train, y_train, scoring="neg_mean_squared_error", cv=10)

In [37]:
tree_rmse_scores = np.sqrt(-scores)
print(tree_rmse_scores)

[67169.56964276 70195.38323593 66343.28457651 70869.63556661
 69063.6548894  66683.32671171 61734.65916625 68896.46343474
 69208.8874542  67923.80137524]


In [38]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

In [39]:
display_scores(tree_rmse_scores)

Scores: [67169.56964276 70195.38323593 66343.28457651 70869.63556661
 69063.6548894  66683.32671171 61734.65916625 68896.46343474
 69208.8874542  67923.80137524]
Mean: 67808.8666053355
Standard deviation: 2459.9565950823185


In [41]:
forest_reg = RandomForestRegressor()

In [42]:
scores = cross_val_score(forest_reg, x_train, y_train, scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-scores)
display_scores(forest_rmse_scores)

Scores: [46746.14947535 50579.74045505 47605.9567768  50365.02962872
 50251.2993067  46974.30315346 45854.98550395 50671.08916695
 49682.37117378 49690.20202693]
Mean: 48842.11266676744
Standard deviation: 1744.352673582337


In [44]:
param_grid = [
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
]

forest_reg = RandomForestRegressor()

grid_search = GridSearchCV(forest_reg, 
                           param_grid, 
                           cv=5,
                           scoring='neg_mean_squared_error',
                           return_train_score=True)
grid_search.fit(x_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

In [45]:
# print out the best params
print(grid_search.best_params_)

{'max_features': 8, 'n_estimators': 30}


In [47]:
# print out the best estimator
print(grid_search.best_estimator_)

RandomForestRegressor(max_features=8, n_estimators=30)


In [48]:
# print out the evaluation scores
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

64266.80853606 {'max_features': 2, 'n_estimators': 3}
54797.86904823695 {'max_features': 2, 'n_estimators': 10}
52211.17198665745 {'max_features': 2, 'n_estimators': 30}
60473.29996112008 {'max_features': 4, 'n_estimators': 3}
52812.75422570901 {'max_features': 4, 'n_estimators': 10}
50187.39350224656 {'max_features': 4, 'n_estimators': 30}
59152.62416062544 {'max_features': 6, 'n_estimators': 3}
51399.20446313793 {'max_features': 6, 'n_estimators': 10}
49735.67035271057 {'max_features': 6, 'n_estimators': 30}
57753.12636576443 {'max_features': 8, 'n_estimators': 3}
52151.52344939643 {'max_features': 8, 'n_estimators': 10}
49640.09365682757 {'max_features': 8, 'n_estimators': 30}
62219.94411316376 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
53998.96568757079 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
59744.69927765355 {'bootstrap': False, 'max_features': 3, 'n_estimators': 3}
52229.39262349927 {'bootstrap': False, 'max_features': 3, 'n_estimators': 10}
5